# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 1:32PM,244176,10,0085937549,ISDIN Corporation,Released
1,Aug 15 2022 1:32PM,244176,10,0085937554,ISDIN Corporation,Released
2,Aug 15 2022 1:32PM,244176,10,0085937573,ISDIN Corporation,Released
3,Aug 15 2022 1:32PM,244176,10,0085937592,ISDIN Corporation,Released
4,Aug 15 2022 1:32PM,244176,10,0085937593,ISDIN Corporation,Released
5,Aug 15 2022 1:32PM,244176,10,0085937601,ISDIN Corporation,Released
6,Aug 15 2022 1:32PM,244176,10,0085937695,ISDIN Corporation,Released
7,Aug 15 2022 1:32PM,244176,10,0085937696,ISDIN Corporation,Released
8,Aug 15 2022 1:32PM,244176,10,0085937700,ISDIN Corporation,Released
9,Aug 15 2022 1:32PM,244176,10,0085937699,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,244172,Released,2
31,244173,Released,12
32,244174,Released,1
33,244175,Released,1
34,244176,Released,28


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244172,NaN,NaN,2.0
244173,NaN,NaN,12.0
244174,NaN,NaN,1.0
244175,NaN,NaN,1.0
244176,NaN,NaN,28.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244122,0.0,0.0,6.0
244130,0.0,1.0,0.0
244135,0.0,0.0,10.0
244137,0.0,0.0,1.0
244139,1.0,7.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244122,0,0,6
244130,0,1,0
244135,0,0,10
244137,0,0,1
244139,1,7,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244122,0,0,6
1,244130,0,1,0
2,244135,0,0,10
3,244137,0,0,1
4,244139,1,7,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244122,,,6
1,244130,,1,
2,244135,,,10
3,244137,,,1
4,244139,1,7,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 1:32PM,244176,10,ISDIN Corporation
28,Aug 15 2022 1:26PM,244175,10,"Senseonics, Incorporated"
29,Aug 15 2022 1:25PM,244174,10,Yusen – 3D Matrix
30,Aug 15 2022 1:24PM,244173,10,"Snap Medical Industries, LLC"
42,Aug 15 2022 1:22PM,244172,10,Emerginnova
44,Aug 15 2022 1:20PM,244171,10,Yusen – 3D Matrix
45,Aug 15 2022 1:16PM,244170,10,Yusen – 3D Matrix
46,Aug 15 2022 12:44PM,244165,20,"Exact-Rx, Inc."
62,Aug 15 2022 12:40PM,244164,19,"GCH Granules USA, Inc."
63,Aug 15 2022 12:26PM,244163,10,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 15 2022 1:32PM,244176,10,ISDIN Corporation,,,28
1,Aug 15 2022 1:26PM,244175,10,"Senseonics, Incorporated",,,1
2,Aug 15 2022 1:25PM,244174,10,Yusen – 3D Matrix,,,1
3,Aug 15 2022 1:24PM,244173,10,"Snap Medical Industries, LLC",,,12
4,Aug 15 2022 1:22PM,244172,10,Emerginnova,,,2
5,Aug 15 2022 1:20PM,244171,10,Yusen – 3D Matrix,,,1
6,Aug 15 2022 1:16PM,244170,10,Yusen – 3D Matrix,,,1
7,Aug 15 2022 12:44PM,244165,20,"Exact-Rx, Inc.",,7,9
8,Aug 15 2022 12:40PM,244164,19,"GCH Granules USA, Inc.",,,1
9,Aug 15 2022 12:26PM,244163,10,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 1:32PM,244176,10,ISDIN Corporation,28,,
1,Aug 15 2022 1:26PM,244175,10,"Senseonics, Incorporated",1,,
2,Aug 15 2022 1:25PM,244174,10,Yusen – 3D Matrix,1,,
3,Aug 15 2022 1:24PM,244173,10,"Snap Medical Industries, LLC",12,,
4,Aug 15 2022 1:22PM,244172,10,Emerginnova,2,,
5,Aug 15 2022 1:20PM,244171,10,Yusen – 3D Matrix,1,,
6,Aug 15 2022 1:16PM,244170,10,Yusen – 3D Matrix,1,,
7,Aug 15 2022 12:44PM,244165,20,"Exact-Rx, Inc.",9,7,
8,Aug 15 2022 12:40PM,244164,19,"GCH Granules USA, Inc.",1,,
9,Aug 15 2022 12:26PM,244163,10,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 1:32PM,244176,10,ISDIN Corporation,28,,
1,Aug 15 2022 1:26PM,244175,10,"Senseonics, Incorporated",1,,
2,Aug 15 2022 1:25PM,244174,10,Yusen – 3D Matrix,1,,
3,Aug 15 2022 1:24PM,244173,10,"Snap Medical Industries, LLC",12,,
4,Aug 15 2022 1:22PM,244172,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 1:32PM,244176,10,ISDIN Corporation,28.0,NaN,NaN
1,Aug 15 2022 1:26PM,244175,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Aug 15 2022 1:25PM,244174,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Aug 15 2022 1:24PM,244173,10,"Snap Medical Industries, LLC",12.0,NaN,NaN
4,Aug 15 2022 1:22PM,244172,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 15 2022 1:32PM,244176,10,ISDIN Corporation,28.0,0.0,0.0
1,Aug 15 2022 1:26PM,244175,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Aug 15 2022 1:25PM,244174,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Aug 15 2022 1:24PM,244173,10,"Snap Medical Industries, LLC",12.0,0.0,0.0
4,Aug 15 2022 1:22PM,244172,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4150733,253.0,28.0,1.0
15,244141,1.0,17.0,3.0
16,488267,1.0,1.0,0.0
19,488319,1.0,1.0,0.0
20,976583,61.0,7.0,0.0
22,732452,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4150733,253.0,28.0,1.0
1,15,244141,1.0,17.0,3.0
2,16,488267,1.0,1.0,0.0
3,19,488319,1.0,1.0,0.0
4,20,976583,61.0,7.0,0.0
5,22,732452,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,253.0,28.0,1.0
1,15,1.0,17.0,3.0
2,16,1.0,1.0,0.0
3,19,1.0,1.0,0.0
4,20,61.0,7.0,0.0
5,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,253.0
1,15,Released,1.0
2,16,Released,1.0
3,19,Released,1.0
4,20,Released,61.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,22
Status,,,,,,
Completed,1.0,3.0,0.0,0.0,0.0,0.0
Executing,28.0,17.0,1.0,1.0,7.0,0.0
Released,253.0,1.0,1.0,1.0,61.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,22
0,Completed,1.0,3.0,0.0,0.0,0.0,0.0
1,Executing,28.0,17.0,1.0,1.0,7.0,0.0
2,Released,253.0,1.0,1.0,1.0,61.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,22
0,Completed,1.0,3.0,0.0,0.0,0.0,0.0
1,Executing,28.0,17.0,1.0,1.0,7.0,0.0
2,Released,253.0,1.0,1.0,1.0,61.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()